<a href="https://colab.research.google.com/github/jswooo/HuggingFace/blob/main/2_4(handling_multiple_sequence).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
'''
sequence를 tensor 형태로 model의 input으로 사용하는 과정 
'''
## 에러나는 코드.
import torch 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence) # 토큰화
ids = tokenizer.convert_tokens_to_ids(tokens) # 숫자로
input_ids = torch.tensor(ids) # 텐서로 

model(input_ids) #모델에 대입 

RuntimeError: ignored

In [ ]:
'''
위에 오류 나는 이유 = transformer model은 다수의 문장을 디폴트로 생각함. tokenizer는 input_ids를 텐서로 바꿀때 차원을 하나 추가함. (우리가 보낸건 그냥 1차원이라서 에러난거)
'''
tokenized_inputs = tokenizer(sequence, return_tensors='pt')
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence) # 토큰화
ids = tokenizer.convert_tokens_to_ids(tokens) # 숫자로
input_ids = torch.tensor(ids) # 텐서로 

input_ids = torch.tensor([ids]) # []로 차원을 하나 추가해줌.. 
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

# 밑에 결과를 확인하면 [[]]로 2차원 텐서로 구성되어 있음을 확인할 수 있다. 

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [ ]:
'''
batching = 한 번에 다수의 sentence를 모델에 보내는 동작 (하나의 문장만으로도 batch를 만들 수 있음) => 다수의 문장에 대해서 모델이 돌아갈 수 있도록 도와줌 
'''
# 동일 문장으로 형성된 batch 
batched_ids = [ids, ids]

input_ids = torch.tensor(batched_ids)

output = model(input_ids)
print("Logits:", output.logits)

Logits: tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [ ]:
'''
pad => tensor를 대입할 때는 직사각형 형태가 필요함. 그러나 문장들의 길이가 달라서 이게 안 됌. 따라서 padding이 필요함. 
'''
# 안되는 예 
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

input_ids = torch.tensor(batched_ids)

ValueError: ignored

In [ ]:
# padding token으로 padding을 진행 
# 간단 예 
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

In [ ]:
# padding 잘못된 예 2 
'''
attention layer에서 padding token에 대한 계산도 들어가기에, 개별 계산과 batched 계산 사이에서 logit 값의 차이가 발생함. 
해결하기 위해서는 pad_token을 무시하라는 내용을 넣어줘야 함! => "attention mask"를 통해서 해결 
'''
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits) # 개별 
print(model(torch.tensor(sequence2_ids)).logits) # 개별 
print(model(torch.tensor(batched_ids)).logits) # batch 

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


### Attention mask 

In [ ]:
'''
Attention mask는 input id 텐서와 동일한 크기의 텐서이다. 0과 1로 구성되어 있다.
0이면 무시해도되는 토큰이고, 1이면 신경써야 하는 토큰의 위치를 의미. (모델의 attention layer에서 무시하거나 주의하는 토큰들의 위치를 의미) 
'''
sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(model(torch.tensor(sequence1_ids)).logits) # 개별 
print(model(torch.tensor(sequence2_ids)).logits) # 개별
print(outputs.logits) # attention mask를 사용한 batch 

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [ ]:
'''
일반적으로 transformer는 512 또는 1024 개의 토큰까지만 처리 가능. 
이외에는 더 긴 시퀀스를 처리하는 모델을 사용하거나, 시퀀스를 max_length 까지만 짤라서 써야.
'''